In [2]:
#loading Github folder to access files for Project
!git clone https://github.com/IS737StockPicker/stonks.git

fatal: destination path 'stonks' already exists and is not an empty directory.


In [3]:
#importing packages 
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer #text data
from sklearn.naive_bayes import MultinomialNB # The Naive Bayes algo


from tqdm import tqdm # progress slider for ``for'' loops

#import CSV from Github Repository
#df = pd.read_csv(git_hub_URL);

In [4]:
#setting up my drive to save the file so I can run later on in a seperate notebook 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Glancing at the file entries
huff_news = pd.read_json('/content/drive/MyDrive/News_Category_Dataset_v3.json', lines = True);
huff_news.head ()
huff_news['date']

0        2022-09-23
1        2022-09-23
2        2022-09-23
3        2022-09-23
4        2022-09-22
            ...    
209522   2012-01-28
209523   2012-01-28
209524   2012-01-28
209525   2012-01-28
209526   2012-01-28
Name: date, Length: 209527, dtype: datetime64[ns]

In [6]:
#Filtering Data to eliminate entries before 03/13/22 and after 09/13/22

huff_news_filtered = huff_news.loc[(huff_news['date'] >= '2022-03-13') & (huff_news['date'] <= '2022-09-13')]
huff_news_filtered


,link,headline,category,short_description,authors,date
56,https://www.huffpost.com/entry/us-inflation-nu...,U.S. Inflation Falls For 2nd Straight Month On...,U.S. NEWS,"On a monthly basis, prices rose 0.1%, after a ...","Christopher Rugaber, AP",2022-09-13
57,https://www.huffpost.com/entry/jean-luc-godard...,"Jean-Luc Godard, Pioneering French Filmmaker, ...",ENTERTAINMENT,The director was best known for the groundbrea...,Matthew Jacobs,2022-09-13
58,https://www.huffpost.com/entry/nurses-strike-m...,Thousands Of Minnesota Nurses Launch 3-day Str...,U.S. NEWS,They're pressing for salary increases they say...,"TRISHA AHMED and GRETCHEN EHLKE, AP",2022-09-13
59,https://www.huffpost.com/entry/biden-cancer-mo...,Joe Biden Urges National Unity In Speech On Re...,POLITICS,"""Cancer does not discriminate red and blue,"" t...",Nick Visser,2022-09-13
60,https://www.huffpost.com/entry/tim-scott-senat...,Sen. Tim Scott Downplays Electability Concerns...,POLITICS,"""Who we have on the field is who we’re gonna p...",Marita Vlachou,2022-09-12
...,...,...,...,...,...,...
1023,https://www.huffpost.com/entry/nfl-quarterback...,Tom Brady Announces He's Not Retiring After Al...,SPORTS,"""That time will come. But it’s not now,"" the N...",Josephine Harvey,2022-03-13
1024,https://www.huffpost.com/entry/nyc-suspect-tar...,NYC Police Search For Suspect Who Shot 2 Sleep...,CRIME,"Surveillance video of the violence is ""chillin...",Mary Papenfuss,2022-03-13
1025,https://www.huffpost.com/entry/united-states-c...,"Census: Black Population Grows In Suburbs, Shr...",U.S. NEWS,"For decades, Black residents have been leaving...","SOPHIA TAREEN, AP",2022-03-13
1026,https://www.huffpost.com/entry/united-states-g...,Average U.S. Gas Price Rises 22% In Two Weeks ...,U.S. NEWS,The price at the pump is $1.54 higher than it ...,,2022-03-13


In [7]:
#Eliminating Unwanted Columns to condense data to be able to upload to Github

#Droping Unwanted Columns: 
unwanted_columns =['link','authors']
huff_news_condensed = huff_news_filtered.drop (unwanted_columns, axis = 1)
 
# Print the new dataframe
huff_news_condensed

,headline,category,short_description,date
56,U.S. Inflation Falls For 2nd Straight Month On...,U.S. NEWS,"On a monthly basis, prices rose 0.1%, after a ...",2022-09-13
57,"Jean-Luc Godard, Pioneering French Filmmaker, ...",ENTERTAINMENT,The director was best known for the groundbrea...,2022-09-13
58,Thousands Of Minnesota Nurses Launch 3-day Str...,U.S. NEWS,They're pressing for salary increases they say...,2022-09-13
59,Joe Biden Urges National Unity In Speech On Re...,POLITICS,"""Cancer does not discriminate red and blue,"" t...",2022-09-13
60,Sen. Tim Scott Downplays Electability Concerns...,POLITICS,"""Who we have on the field is who we’re gonna p...",2022-09-12
...,...,...,...,...
1023,Tom Brady Announces He's Not Retiring After Al...,SPORTS,"""That time will come. But it’s not now,"" the N...",2022-03-13
1024,NYC Police Search For Suspect Who Shot 2 Sleep...,CRIME,"Surveillance video of the violence is ""chillin...",2022-03-13
1025,"Census: Black Population Grows In Suburbs, Shr...",U.S. NEWS,"For decades, Black residents have been leaving...",2022-03-13
1026,Average U.S. Gas Price Rises 22% In Two Weeks ...,U.S. NEWS,The price at the pump is $1.54 higher than it ...,2022-03-13


In [8]:
#Converting Json to CSV to Upload to github for final notebook code & so I dont have to mount drive in second notebook
huff_news_condensed.to_csv ('Huff_News_Condensed.csv', index = False)

In [9]:
#reset code to prevent errors when running the code again from scratch
%rm -rf stonks/